In [10]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.docstore.document import Document

# Use Gemma:2B as the Ollama embedding model
embeddings = OllamaEmbeddings(model="gemma:2b")

# ✅ Updated Sample Data (Tea Business + AI & Consumer Behavior)
sample_data = [
    "Tea culture is growing rapidly as consumers seek healthier alternatives to coffee.",
    "AI-powered customer service is revolutionizing how tea businesses handle inquiries.",
    "The best-selling tea flavors in 2025 are matcha, chamomile, and oolong.",
    "Consumer psychology suggests that branding and packaging influence tea purchasing decisions.",
    "Subscription-based tea services are on the rise due to personalized recommendations.",
    "AI-driven ad campaigns help tea brands target health-conscious consumers.",
    "Premium loose-leaf teas are gaining popularity over traditional tea bags.",
    "Social media marketing plays a key role in promoting specialty tea brands.",
]

# Convert sample data into LangChain Document format
docs = [Document(page_content=text) for text in sample_data]

# Create FAISS vector store with sample data
vectorstoreDB = FAISS.from_documents(docs, embeddings)

print("FAISS index created with test data!")

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOllama
from langchain.schema.runnable import RunnableMap

# ✅ Initialize Ollama LLM (Using Gemma:2B)
llm = ChatOllama(model="gemma:2b")

# ✅ Define the Retrieval-Augmented Generation (RAG) Prompt
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
        {context}
    </context>

    Question: {question}
    Answer:
    """
)

# ✅ Create Document Chain (Combining Retrieved Docs into Final Response)
document_chain = create_stuff_documents_chain(llm, prompt)

# ✅ Create Retriever from FAISS
retriever = vectorstoreDB.as_retriever()

# ✅ Build the RAG Pipeline
retrieval_chain = RunnableMap({
    "context": retriever,   # Fetch relevant documents
    "question": lambda x: x["query"],  # Extract question
}) | document_chain  # Pass results to document chain

# ✅ Example Query
question = "What are the latest trends in the tea industry?"
response = retrieval_chain.invoke({"query": question})  # Use `.invoke()`

# ✅ Print the AI-Generated Response
print("Generated Answer:")
print(response)


FAISS index created with test data!
Generated Answer:
Sure, here's the answer to the question:

The latest trends in the tea industry are:

- Matcha, chamomile, and oolong are the best-selling tea flavors.

- Subscription-based tea services are on the rise due to personalized recommendations.

- AI-driven ad campaigns help tea brands target health-conscious consumers.

- AI-powered customer service is revolutionizing how tea businesses handle inquiries.
